<a href="https://colab.research.google.com/github/EzraMW/ML/blob/main/Transfer_Learning_for_Newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using pre-trained word embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/05<br>
**Last modified:** 2020/05/05<br>
**Description:** Text classification on the Newsgroup20 dataset using pre-trained GloVe word embeddings.

Taken from: 
https://keras.io/examples/nlp/pretrained_word_embeddings/

Modified by Avi Rosenfeld on 2023/21/02

## Setup

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization, Embedding
from sklearn.datasets import fetch_20newsgroups
import gensim.downloader as api

In [30]:
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
glove_model = api.load("glove-wiki-gigaword-200")

## Download the Newsgroup20 data

Note that as opposed to our previous notebook, here we download the entire 20_newsgroup data.

In [3]:
newsgroups_all = fetch_20newsgroups(subset='all', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
newsgroups_all_headers = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

In [37]:
def transfer_learning(model, d, size, stop_words=None, words=None):
  embedding_dim = model.vector_size 
  print("Embedding Dimension: ", embedding_dim)

  samples = d.data
  labels =  d.target
  class_names = d.target_names

  seed = 1337
  rng = np.random.RandomState(seed)
  rng.shuffle(samples)
  rng = np.random.RandomState(seed)
  rng.shuffle(labels)

  # Extract a training & validation split
  validation_split = 0.2
  num_validation_samples = int(validation_split * len(samples))
  train_samples = samples[:-num_validation_samples]
  val_samples = samples[-num_validation_samples:]
  train_labels = labels[:-num_validation_samples]
  val_labels = labels[-num_validation_samples:]

  vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=size)
  text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
  vectorizer.adapt(text_ds)

  voc = vectorizer.get_vocabulary()
  
  # if inputted just use these specific words for the vocabulary
  if words != None:
    voc = words

  word_index = dict(zip(voc, range(len(voc))))

  vocab_size = len(word_index)
  num_tokens = len(voc) + 2
  embedding_matrix = np.zeros((num_tokens, embedding_dim))

  hits = 0
  misses = 0
          
  # Prepare embedding matrix
  if stop_words == None:
    for word, i in word_index.items():
        if word in model:
          hits += 1
          embedding_matrix[i] = model[word]
        else:
            misses += 1
  else:
    for word, i in word_index.items():
        if word in model and word not in stop_words:
          hits += 1
          embedding_matrix[i] = model[word]
        else:
            misses += 1
  print("Converted %d words (%d misses)" % (hits, misses))

  embedding_layer = Embedding(
      num_tokens,
      embedding_dim,
      embeddings_initializer=keras.initializers.Constant(embedding_matrix),
      trainable=False,
  ) 

  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(128, 5, activation="relu")(x)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(128, 5, activation="relu")(x)
  x = layers.GlobalMaxPooling1D()(x)
  x = layers.Dense(128, activation="relu")(x)
  x = layers.Dropout(0.5)(x)
  preds = layers.Dense(len(class_names), activation="softmax")(x)
  model = keras.Model(int_sequences_input, preds)
  # print(model.summary())

  x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
  x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

  y_train = np.array(train_labels)
  y_val = np.array(val_labels)

  model.compile(
      loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"],
  )
  print("Running Model:")
  model.fit(x_train, y_train, batch_size=128, epochs=30, validation_data=(x_val, y_val))

  string_input = keras.Input(shape=(1,), dtype="string")
  x = vectorizer(string_input)
  preds = model(x)
  end_to_end_model = keras.Model(string_input, preds)
  return end_to_end_model

Without Headers, Footers, and Quotes

In [7]:
transfer_learning(glove_model, newsgroups_all_headers, 200)

Embedding Dimension:  200
Converted 17655 words (2345 misses)
Running Model:
Epoch 1/30
118/118 [==============================] - 14s 18ms/step - loss: 2.6302 - acc: 0.1478 - val_loss: 1.9788 - val_acc: 0.2966
Epoch 2/30
118/118 [==============================] - 1s 12ms/step - loss: 1.7372 - acc: 0.3976 - val_loss: 1.1859 - val_acc: 0.6007
Epoch 3/30
118/118 [==============================] - 1s 12ms/step - loss: 1.1511 - acc: 0.6100 - val_loss: 0.9612 - val_acc: 0.6702
Epoch 4/30
118/118 [==============================] - 1s 12ms/step - loss: 0.8237 - acc: 0.7153 - val_loss: 0.7552 - val_acc: 0.7437
Epoch 5/30
118/118 [==============================] - 1s 12ms/step - loss: 0.6000 - acc: 0.7920 - val_loss: 0.6881 - val_acc: 0.7623
Epoch 6/30
118/118 [==============================] - 1s 12ms/step - loss: 0.4466 - acc: 0.8476 - val_loss: 0.6523 - val_acc: 0.8023
Epoch 7/30
118/118 [==============================] - 2s 15ms/step - loss: 0.3214 - acc: 0.8947 - val_loss: 0.6186 - val_acc

The model did very well with the headers etc. removed. It got a training accuracy of ~ 99% and a validation accuracy around 85%.

With Headers, Footers, and Quotes

In [6]:
transfer_learning(glove_model, newsgroups_all, 200)

Embedding Dimension:  200
Converted 18326 words (1674 misses)
Running Model:
Epoch 1/30
118/118 [==============================] - 15s 20ms/step - loss: 2.7186 - acc: 0.1244 - val_loss: 2.1701 - val_acc: 0.2674
Epoch 2/30
118/118 [==============================] - 1s 12ms/step - loss: 1.9587 - acc: 0.3299 - val_loss: 1.5963 - val_acc: 0.4585
Epoch 3/30
118/118 [==============================] - 1s 12ms/step - loss: 1.5213 - acc: 0.4858 - val_loss: 1.4007 - val_acc: 0.5357
Epoch 4/30
118/118 [==============================] - 1s 12ms/step - loss: 1.2890 - acc: 0.5610 - val_loss: 1.3553 - val_acc: 0.5511
Epoch 5/30
118/118 [==============================] - 1s 12ms/step - loss: 1.1278 - acc: 0.6174 - val_loss: 1.3277 - val_acc: 0.5627
Epoch 6/30
118/118 [==============================] - 1s 12ms/step - loss: 0.9781 - acc: 0.6618 - val_loss: 1.2905 - val_acc: 0.5869
Epoch 7/30
118/118 [==============================] - 1s 12ms/step - loss: 0.8328 - acc: 0.7097 - val_loss: 1.3486 - val_acc

However, when we left in the headers, footers and quotes the training accuracy slightly descresed (suprisingly) but the validation accuracy dramatically decreased to less than 60%. This demonstrates the tremendous amount of overfitting happening due to leaving in the headers, footers, and quotes

So, going forward, we will remove the headers, footers, and quotes from the data

# Improve Performance for 4 Categories

In [6]:
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# newsgroups_cats_header = fetch_20newsgroups(subset='all',categories=cats)
newsgroups_cats = fetch_20newsgroups(subset='all',categories=cats, remove=('headers', 'footers', 'quotes'))

Let's establish the baseline metric using the glove model with these 4 categories

In [13]:
transfer_learning(glove_model, newsgroups_cats, 200)

Embedding Dimension:  200
Converted 15881 words (4119 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 2s 27ms/step - loss: 1.3292 - acc: 0.3513 - val_loss: 1.1394 - val_acc: 0.4697
Epoch 2/30
22/22 [==============================] - 0s 15ms/step - loss: 0.9427 - acc: 0.5760 - val_loss: 0.8783 - val_acc: 0.6027
Epoch 3/30
22/22 [==============================] - 0s 12ms/step - loss: 0.7335 - acc: 0.6705 - val_loss: 0.6603 - val_acc: 0.6425
Epoch 4/30
22/22 [==============================] - 0s 16ms/step - loss: 0.5972 - acc: 0.7207 - val_loss: 0.6617 - val_acc: 0.6987
Epoch 5/30
22/22 [==============================] - 0s 16ms/step - loss: 0.5328 - acc: 0.7358 - val_loss: 0.5895 - val_acc: 0.7149
Epoch 6/30
22/22 [==============================] - 0s 16ms/step - loss: 0.4577 - acc: 0.7756 - val_loss: 0.7352 - val_acc: 0.6529
Epoch 7/30
22/22 [==============================] - 0s 15ms/step - loss: 0.4186 - acc: 0.8052 - val_loss: 0.6240 - val_acc: 0.7356
Epoch 

We acheived an over 97% training accuracy and over 75% validation accuracy. This isn't as good as we got using all 20 categories but very solid

Let's try tansfering the training from different models

In [41]:
glove_model_300 = api.load("glove-wiki-gigaword-300")

In [10]:
transfer_learning(glove_model_300, newsgroups_cats, 300)

Embedding Dimension:  300
Converted 16166 words (3834 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 3s 41ms/step - loss: 1.3433 - acc: 0.3439 - val_loss: 1.1421 - val_acc: 0.5066
Epoch 2/30
22/22 [==============================] - 0s 20ms/step - loss: 0.9240 - acc: 0.5897 - val_loss: 0.7450 - val_acc: 0.6677
Epoch 3/30
22/22 [==============================] - 0s 22ms/step - loss: 0.6681 - acc: 0.6882 - val_loss: 0.6742 - val_acc: 0.6839
Epoch 4/30
22/22 [==============================] - 0s 22ms/step - loss: 0.5814 - acc: 0.7085 - val_loss: 0.5839 - val_acc: 0.7297
Epoch 5/30
22/22 [==============================] - 0s 22ms/step - loss: 0.4875 - acc: 0.7598 - val_loss: 0.5850 - val_acc: 0.7164
Epoch 6/30
22/22 [==============================] - 1s 24ms/step - loss: 0.4117 - acc: 0.8037 - val_loss: 0.6149 - val_acc: 0.7312
Epoch 7/30
22/22 [==============================] - 0s 22ms/step - loss: 0.3543 - acc: 0.8421 - val_loss: 0.6053 - val_acc: 0.7489
Epoch 

Very solid results but not very different than the glove model

In [7]:
w2v_model = api.load("word2vec-google-news-300")

In [8]:
transfer_learning(w2v_model, newsgroups_cats, 300)

Embedding Dimension:  300
Converted 14143 words (5857 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 13s 46ms/step - loss: 1.3032 - acc: 0.3572 - val_loss: 1.0284 - val_acc: 0.5569
Epoch 2/30
22/22 [==============================] - 1s 24ms/step - loss: 0.8367 - acc: 0.6262 - val_loss: 0.7207 - val_acc: 0.6396
Epoch 3/30
22/22 [==============================] - 1s 23ms/step - loss: 0.6783 - acc: 0.6845 - val_loss: 0.6192 - val_acc: 0.7031
Epoch 4/30
22/22 [==============================] - 1s 24ms/step - loss: 0.5556 - acc: 0.7362 - val_loss: 0.5867 - val_acc: 0.7001
Epoch 5/30
22/22 [==============================] - 1s 26ms/step - loss: 0.4642 - acc: 0.7731 - val_loss: 0.6013 - val_acc: 0.6765
Epoch 6/30
22/22 [==============================] - 1s 27ms/step - loss: 0.4117 - acc: 0.8148 - val_loss: 0.5896 - val_acc: 0.7046
Epoch 7/30
22/22 [==============================] - 1s 32ms/step - loss: 0.3250 - acc: 0.8594 - val_loss: 0.5969 - val_acc: 0.7061
Epoch

This also didn't really improve on the previous model

## Remove Stop Words

Let's try removing stop words from the dataset and see if that helps

In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [12]:
transfer_learning(glove_model, newsgroups_cats, 200, ENGLISH_STOP_WORDS)

Embedding Dimension:  200
Converted 15858 words (4142 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 3s 36ms/step - loss: 1.2935 - acc: 0.3745 - val_loss: 0.9862 - val_acc: 0.6278
Epoch 2/30
22/22 [==============================] - 0s 19ms/step - loss: 0.8858 - acc: 0.6185 - val_loss: 0.7313 - val_acc: 0.6455
Epoch 3/30
22/22 [==============================] - 1s 24ms/step - loss: 0.6763 - acc: 0.6768 - val_loss: 0.6237 - val_acc: 0.7061
Epoch 4/30
22/22 [==============================] - 0s 17ms/step - loss: 0.5691 - acc: 0.7162 - val_loss: 0.6188 - val_acc: 0.6987
Epoch 5/30
22/22 [==============================] - 0s 15ms/step - loss: 0.4977 - acc: 0.7517 - val_loss: 0.5745 - val_acc: 0.7105
Epoch 6/30
22/22 [==============================] - 0s 14ms/step - loss: 0.4277 - acc: 0.7856 - val_loss: 0.5791 - val_acc: 0.7223
Epoch 7/30
22/22 [==============================] - 0s 12ms/step - loss: 0.3763 - acc: 0.8133 - val_loss: 0.6253 - val_acc: 0.7134
Epoch 

Other than the last epoch, which seems like an outlier, we got a training accuracy above 97% and a validation accuracy around 78%. This is slightly better than our baseline model.

## Use the top CHI2 Words

In [32]:
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(newsgroups_cats.data)
chi2score = chi2(vectors, newsgroups_cats.target)[0]
wscores = zip(vectorizer.get_feature_names_out(),chi2score)
wchi2 = sorted(wscores,key=lambda x:x[1],reverse = True) 

In [36]:
chi2_words = [t[0] for t in wchi2]
top_chi2_words = chi2_words[:5000]

In [38]:
transfer_learning(glove_model, newsgroups_cats, 200, words=top_chi2_words)

Embedding Dimension:  200
Converted 4667 words (333 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 3s 33ms/step - loss: 1.4144 - acc: 0.2974 - val_loss: 1.3542 - val_acc: 0.3264
Epoch 2/30
22/22 [==============================] - 0s 18ms/step - loss: 1.3683 - acc: 0.3018 - val_loss: 1.3485 - val_acc: 0.3353
Epoch 3/30
22/22 [==============================] - 0s 18ms/step - loss: 1.3471 - acc: 0.3432 - val_loss: 1.3270 - val_acc: 0.3456
Epoch 4/30
22/22 [==============================] - 0s 15ms/step - loss: 1.2911 - acc: 0.3823 - val_loss: 1.3381 - val_acc: 0.3929
Epoch 5/30
22/22 [==============================] - 0s 15ms/step - loss: 1.2401 - acc: 0.4210 - val_loss: 1.2105 - val_acc: 0.4564
Epoch 6/30
22/22 [==============================] - 0s 13ms/step - loss: 1.0583 - acc: 0.5391 - val_loss: 1.1264 - val_acc: 0.4727
Epoch 7/30
22/22 [==============================] - 0s 13ms/step - loss: 0.8849 - acc: 0.6063 - val_loss: 1.0465 - val_acc: 0.5362
Epoch 8/

Ouch! This significantly reduced the validation accuracy

In [40]:
transfer_learning(w2v_model, newsgroups_cats, 300, words=top_chi2_words)

Embedding Dimension:  300
Converted 4102 words (898 misses)
Running Model:
Epoch 1/30
22/22 [==============================] - 2s 34ms/step - loss: 1.3850 - acc: 0.3018 - val_loss: 1.3479 - val_acc: 0.3368
Epoch 2/30
22/22 [==============================] - 0s 21ms/step - loss: 1.3673 - acc: 0.2923 - val_loss: 1.3389 - val_acc: 0.3516
Epoch 3/30
22/22 [==============================] - 0s 21ms/step - loss: 1.3547 - acc: 0.3221 - val_loss: 1.3289 - val_acc: 0.3530
Epoch 4/30
22/22 [==============================] - 0s 20ms/step - loss: 1.3097 - acc: 0.3620 - val_loss: 1.3004 - val_acc: 0.3767
Epoch 5/30
22/22 [==============================] - 0s 21ms/step - loss: 1.2145 - acc: 0.4328 - val_loss: 1.2025 - val_acc: 0.4461
Epoch 6/30
22/22 [==============================] - 0s 20ms/step - loss: 1.0246 - acc: 0.5199 - val_loss: 1.3102 - val_acc: 0.3973
Epoch 7/30
22/22 [==============================] - 0s 19ms/step - loss: 0.8586 - acc: 0.5963 - val_loss: 1.0271 - val_acc: 0.5273
Epoch 8/

The word2vec model did slightly better than glove but significantly underperformed the baseline

## Different Categories 

These are the suggested categories which are very different from one another

Again, we will remove the headers, footers, and quotes to get the better results and better compare to previous iterations

In [21]:
diff_cats = ['talk.religion.misc', 'comp.sys.ibm.pc.hardware', 'sci.space']
newsgroups_diff_cats = fetch_20newsgroups(subset='all',categories=diff_cats, remove=('headers', 'footers', 'quotes'))

In [22]:
transfer_learning(glove_model, newsgroups_diff_cats, 200)

Embedding Dimension:  200
Converted 15764 words (4236 misses)
Running Model:
Epoch 1/30
17/17 [==============================] - 3s 48ms/step - loss: 1.0969 - acc: 0.3898 - val_loss: 1.0071 - val_acc: 0.5491
Epoch 2/30
17/17 [==============================] - 0s 16ms/step - loss: 0.8303 - acc: 0.6121 - val_loss: 0.5460 - val_acc: 0.7842
Epoch 3/30
17/17 [==============================] - 0s 16ms/step - loss: 0.4818 - acc: 0.8041 - val_loss: 0.3596 - val_acc: 0.8497
Epoch 4/30
17/17 [==============================] - 0s 14ms/step - loss: 0.3094 - acc: 0.8758 - val_loss: 0.3016 - val_acc: 0.8844
Epoch 5/30
17/17 [==============================] - 0s 16ms/step - loss: 0.2162 - acc: 0.9148 - val_loss: 0.3341 - val_acc: 0.8651
Epoch 6/30
17/17 [==============================] - 0s 19ms/step - loss: 0.1765 - acc: 0.9317 - val_loss: 0.2621 - val_acc: 0.8998
Epoch 7/30
17/17 [==============================] - 0s 19ms/step - loss: 0.1203 - acc: 0.9577 - val_loss: 0.2840 - val_acc: 0.9017
Epoch 

WOW! These results are much better. The validation broke 90% accuracy very quickly and even the training accuracy is slightly better

In [23]:
transfer_learning(glove_model, newsgroups_diff_cats, 200, ENGLISH_STOP_WORDS)

Embedding Dimension:  200
Converted 15444 words (4556 misses)
Running Model:
Epoch 1/30
17/17 [==============================] - 2s 32ms/step - loss: 1.0261 - acc: 0.4692 - val_loss: 0.7396 - val_acc: 0.7052
Epoch 2/30
17/17 [==============================] - 0s 17ms/step - loss: 0.5490 - acc: 0.7733 - val_loss: 0.3482 - val_acc: 0.8690
Epoch 3/30
17/17 [==============================] - 0s 14ms/step - loss: 0.3072 - acc: 0.8730 - val_loss: 0.2595 - val_acc: 0.9037
Epoch 4/30
17/17 [==============================] - 0s 13ms/step - loss: 0.2106 - acc: 0.9192 - val_loss: 0.2224 - val_acc: 0.8960
Epoch 5/30
17/17 [==============================] - 0s 16ms/step - loss: 0.1506 - acc: 0.9432 - val_loss: 0.1960 - val_acc: 0.9075
Epoch 6/30
17/17 [==============================] - 0s 13ms/step - loss: 0.1127 - acc: 0.9591 - val_loss: 0.1993 - val_acc: 0.9056
Epoch 7/30
17/17 [==============================] - 0s 20ms/step - loss: 0.0811 - acc: 0.9687 - val_loss: 0.1961 - val_acc: 0.9133
Epoch 

The stopwords don't seem to make much of a difference, but this also did really well. Maybe the irrelevance of the stopwords is due to the distinction between the different categories so the model has an easier time determining which words are most relevant

In [24]:
transfer_learning(w2v_model, newsgroups_diff_cats, 300, ENGLISH_STOP_WORDS)

Embedding Dimension:  300
Converted 13823 words (6177 misses)
Running Model:
Epoch 1/30
17/17 [==============================] - 3s 67ms/step - loss: 0.9850 - acc: 0.4986 - val_loss: 0.7036 - val_acc: 0.6782
Epoch 2/30
17/17 [==============================] - 0s 23ms/step - loss: 0.4753 - acc: 0.8041 - val_loss: 0.3040 - val_acc: 0.8671
Epoch 3/30
17/17 [==============================] - 0s 21ms/step - loss: 0.2690 - acc: 0.8961 - val_loss: 0.3145 - val_acc: 0.8651
Epoch 4/30
17/17 [==============================] - 0s 27ms/step - loss: 0.2316 - acc: 0.9167 - val_loss: 0.2532 - val_acc: 0.8921
Epoch 5/30
17/17 [==============================] - 0s 30ms/step - loss: 0.1292 - acc: 0.9538 - val_loss: 0.2661 - val_acc: 0.8940
Epoch 6/30
17/17 [==============================] - 0s 26ms/step - loss: 0.0952 - acc: 0.9620 - val_loss: 0.2592 - val_acc: 0.8940
Epoch 7/30
17/17 [==============================] - 0s 23ms/step - loss: 0.0706 - acc: 0.9687 - val_loss: 0.2698 - val_acc: 0.8940
Epoch 

Interestingly, the word2vec model did slightly worse on the validation accuracy than the glove model

The word2vec model didn't make much headways either

## Similar Data

In [25]:
same_cats = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware','comp.windows.x']
newsgroups_same_cats = fetch_20newsgroups(subset='all',categories=same_cats, remove=('headers', 'footers', 'quotes'))

In [26]:
transfer_learning(glove_model, newsgroups_same_cats, 200)

Embedding Dimension:  200
Converted 12007 words (7993 misses)
Running Model:
Epoch 1/30
31/31 [==============================] - 3s 31ms/step - loss: 1.6234 - acc: 0.2129 - val_loss: 1.5986 - val_acc: 0.2587
Epoch 2/30
31/31 [==============================] - 0s 15ms/step - loss: 1.5490 - acc: 0.3003 - val_loss: 1.3950 - val_acc: 0.4151
Epoch 3/30
31/31 [==============================] - 0s 15ms/step - loss: 1.3485 - acc: 0.3736 - val_loss: 1.2666 - val_acc: 0.4540
Epoch 4/30
31/31 [==============================] - 0s 15ms/step - loss: 1.1747 - acc: 0.4792 - val_loss: 1.1157 - val_acc: 0.5072
Epoch 5/30
31/31 [==============================] - 0s 16ms/step - loss: 0.9825 - acc: 0.5735 - val_loss: 1.0135 - val_acc: 0.5736
Epoch 6/30
31/31 [==============================] - 0s 14ms/step - loss: 0.8054 - acc: 0.6611 - val_loss: 1.0167 - val_acc: 0.5930
Epoch 7/30
31/31 [==============================] - 0s 13ms/step - loss: 0.6581 - acc: 0.7271 - val_loss: 1.0728 - val_acc: 0.5900
Epoch 

As the parallel opposite conclusion of the different categories, the model had a very hard time distinguishing between the similar categories - only acheiving a validation accuracy of 65%.

In [27]:
transfer_learning(glove_model, newsgroups_same_cats, 200, ENGLISH_STOP_WORDS)

Embedding Dimension:  200
Converted 11770 words (8230 misses)
Running Model:
Epoch 1/30
31/31 [==============================] - 2s 23ms/step - loss: 1.6128 - acc: 0.2331 - val_loss: 1.5597 - val_acc: 0.3241
Epoch 2/30
31/31 [==============================] - 0s 13ms/step - loss: 1.4109 - acc: 0.3550 - val_loss: 1.2876 - val_acc: 0.3885
Epoch 3/30
31/31 [==============================] - 0s 13ms/step - loss: 1.1920 - acc: 0.4633 - val_loss: 1.1487 - val_acc: 0.5051
Epoch 4/30
31/31 [==============================] - 0s 13ms/step - loss: 1.0059 - acc: 0.5686 - val_loss: 1.0905 - val_acc: 0.5460
Epoch 5/30
31/31 [==============================] - 0s 15ms/step - loss: 0.8028 - acc: 0.6708 - val_loss: 1.0353 - val_acc: 0.5879
Epoch 6/30
31/31 [==============================] - 1s 18ms/step - loss: 0.6142 - acc: 0.7557 - val_loss: 0.9798 - val_acc: 0.6207
Epoch 7/30
31/31 [==============================] - 1s 24ms/step - loss: 0.4325 - acc: 0.8423 - val_loss: 1.0475 - val_acc: 0.6431
Epoch 

Removing the stopwords helped slightly

In [28]:
transfer_learning(w2v_model, newsgroups_same_cats, 200, ENGLISH_STOP_WORDS)

Embedding Dimension:  300
Converted 10149 words (9851 misses)
Running Model:
Epoch 1/30
31/31 [==============================] - 3s 27ms/step - loss: 1.6060 - acc: 0.2295 - val_loss: 1.5854 - val_acc: 0.2975
Epoch 2/30
31/31 [==============================] - 1s 16ms/step - loss: 1.4309 - acc: 0.3754 - val_loss: 1.2869 - val_acc: 0.4601
Epoch 3/30
31/31 [==============================] - 0s 15ms/step - loss: 1.1264 - acc: 0.5101 - val_loss: 1.1303 - val_acc: 0.5266
Epoch 4/30
31/31 [==============================] - 0s 16ms/step - loss: 0.8882 - acc: 0.6514 - val_loss: 1.0333 - val_acc: 0.5910
Epoch 5/30
31/31 [==============================] - 1s 17ms/step - loss: 0.6686 - acc: 0.7565 - val_loss: 0.9640 - val_acc: 0.6329
Epoch 6/30
31/31 [==============================] - 1s 18ms/step - loss: 0.5387 - acc: 0.8127 - val_loss: 1.0860 - val_acc: 0.6176
Epoch 7/30
31/31 [==============================] - 1s 18ms/step - loss: 0.3539 - acc: 0.8845 - val_loss: 1.1468 - val_acc: 0.6299
Epoch 

The word2vec model acted quite similarly to the glove model

## Conclusion

Overall, there was not much substantial change between different iterations of transfer learning. The biggest differences came from the similarity of the categories in the dataset. Also, when trained on all 20 categories the model also did better. But adding in the top chi2 words, using different models, removing stopwords all didn't significantly improve the model.